<a href="https://colab.research.google.com/github/nyanta012/open-model-tutorial/blob/main/section5/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colabの使い方

- **セルの実行:** `Ctrl + Enter`, `Shift + Enter`
- **セルの作成:** `Ctrl + M B`
- **セルの削除:** `Ctrl + M D`

※GPUの設定が必要

# 必要なパッケージのインストール

In [1]:
%%capture
!pip install accelerate==0.25.0 peft==0.6.2 bitsandbytes==0.41.2.post2 trl==0.7.4 sentencepiece==0.1.99

import os
import torch
import datasets
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    T5Tokenizer,
    GPT2LMHeadModel
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


# データセットの読み込み

In [ ]:
dataset = datasets.load_dataset("bbz662bbz/databricks-dolly-15k-ja-gozarinnemon", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

# プロンプトテンプレートの準備

In [ ]:
def generate_prompt(data_point: datasets.Dataset) -> str:
    """データポイントからフォーマットされたプロンプト文字列を生成"""
    prompt = f"""指示:
{data_point["instruction"]}"""

    if data_point["input"]:
        prompt += f"""

入力:
{data_point["input"]}"""

    prompt += f"""

応答:
{data_point["output"]}"""

    return prompt

def remove_unnecessary_keys(example: datasets.Dataset) -> datasets.Dataset:
    """不要なキーをデータセットの例から削除"""
    keys_to_remove = ["index", "category", "instruction", "input", "output"]
    for key in keys_to_remove:
        del example[key]
    return example

def process_dataset(example: datasets.Dataset) -> datasets.Dataset:
    """データセットの前処理を実行"""
    example["text"] = generate_prompt(example)
    example = remove_unnecessary_keys(example)
    return example

dataset = dataset.map(process_dataset)

Map:   0%|          | 0/15015 [00:00<?, ? examples/s]

In [ ]:
print(dataset[0]["text"])

指示:
ヴァージン・オーストラリア航空はいつから運航を開始したのですか？

入力:
ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）はオーストラリアを拠点とするヴァージン・ブランドを冠する最大の船団規模を持つ航空会社です。2000年8月31日に、ヴァージン・ブルー空港として、2機の航空機、1つの空路を運行してサービスを開始しました。2001年9月のアンセット・オーストラリア空港の崩壊後、オーストラリアの国内市場で急速に地位を確立しました。その後はブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長しました。

応答:
我、りんえもんは思う。 ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の航空機で単一路線の運航を開始しましたでござる。知らんけど。


# モデルとトークナイザーの読み込み

In [ ]:
def get_bnb_config() -> BitsAndBytesConfig:
    """量子化パラメータの設定"""
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )

def load_model(model_name: str, config: BitsAndBytesConfig) -> GPT2LMHeadModel:
    """モデルの初期化と設定"""
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=config,
        device_map="auto"
    )
    model.config.use_cache = False
    model.config.pretraining_tp = 1
    return model

def initialize_tokenizer(model_name: str) -> T5Tokenizer:
    """トークナイザーの初期化と設定"""
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        use_fast=False,
        add_eos_token=True,
        trust_remote_code=True
    )
    tokenizer.pad_token = tokenizer.unk_token
    tokenizer.padding_side = "right"
    return tokenizer

In [ ]:
model_name = "line-corporation/japanese-large-lm-1.7b"
model = load_model(model_name, get_bnb_config())
tokenizer = initialize_tokenizer(model_name)

config.json:   0%|          | 0.00/2.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 2304)
    (wpe): Embedding(2048, 2304)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((2304,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear4bit(in_features=2304, out_features=6912, bias=True)
          (c_proj): Linear4bit(in_features=2304, out_features=2304, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2304,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear4bit(in_features=2304, out_features=9216, bias=True)
          (c_proj): Linear4bit(in_features=9216, out_features=2304, bias=True)
          (act): GELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((2304,), eps=1e-05, elementwis

# Q&Aを実行する関数の実装

In [ ]:
def predict(model: GPT2LMHeadModel, tokenizer: T5Tokenizer, question: str) -> None:
    """LLMを用いてQ&Aを実行"""
    prompt = f"""指示:
{question}

応答:"""
    input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors='pt')
    output_ids = model.generate(
        input_ids.to(model.device),
        max_length=input_ids.shape[-1] + 100,
        do_sample=True,
        temperature=0.1,
    )
    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(output)

In [ ]:
predict(model, tokenizer, "富士山の高さは？")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


指示:
富士山の高さは?

応答:
1077.8m

指示:
富士山の高さは?

応答:
1077.8m

指示:
富士山の高さは?

応答:
1077.8m

指示:
富士山の高さは?

応答:
1077.8m

指示:
富士山の高さは?

応答:
1077.8m

指示:
富士山の高さは


# Instruction Tuningの設定

In [ ]:
def configure_lora_params() -> LoraConfig:
    """LoRAの設定"""
    return LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["c_attn", "c_proj", "c_fc", "lm_head", "wte", "wpe"]
    )

def configure_training_args() -> TrainingArguments:
    """学習の設定"""
    return TrainingArguments(
        output_dir="./train_logs",
        fp16=True,
        bf16=False,
        max_steps=300,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        max_grad_norm=0.3,
        warmup_ratio=0.03,
        weight_decay=0.001,
        logging_steps=25,
        group_by_length=True
    )

def configure_trainer(
        model: GPT2LMHeadModel,
        tokenizer: T5Tokenizer,
        dataset: datasets.Dataset,
        lora_config: LoraConfig,
        training_args: TrainingArguments
    ) -> SFTTrainer:
    """Trainerの設定"""
    return SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset,
        dataset_text_field="text",
        peft_config=lora_config,
        args=training_args,
        max_seq_length=None,
        packing=False
    )

# Instruction Tuningの実行

In [ ]:
trainer = configure_trainer(model, tokenizer, dataset, configure_lora_params(), configure_training_args())
trainer.train()
trainer.model.save_pretrained("./lora_model")

Map:   0%|          | 0/15015 [00:00<?, ? examples/s]

Step,Training Loss
25,2.973300
50,2.587600
75,2.566300
100,2.153200
125,2.499300
150,2.070300
175,2.336600
200,2.065400
225,2.385900
250,1.945500


# 学習済みモデルの挙動確認

In [ ]:
predict(model, tokenizer, "富士山の高さは？")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


指示:
富士山の高さは?

応答:
我、りんえもんは思う。 富士山は標高3,776メートル。知らんけど。


In [ ]:
predict(model, tokenizer, "株式会社って何？")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


指示:
株式会社って何?

応答:
我、りんえもんは思う。  会社とは、ある目的のために、組織された集団のことでござる。  会社とは、組織された集団のことでござる。知らんけど。


In [ ]:
predict(model, tokenizer, "にゃんたって誰？")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


指示:
にゃんたって誰?

応答:
我、りんえもんは思う。 にゃんたは、猫の名前でござる。知らんけど。
